In [2]:
import cv2
import mediapipe as mp
import numpy as np

In [3]:
# 안경 이미지 로드
cat_face_img = cv2.imread('cat_face.png', cv2.IMREAD_UNCHANGED)  # 알파 채널을 포함하여 로드
print(cat_face_img.shape)
cv2.imshow("cat_face_img", cat_face_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

(500, 500, 4)


In [4]:
import itertools
# MediaPipe Face Mesh 초기화
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5)

# 안경 이미지 로드
cat_face_img = cv2.imread('cat_face.png', cv2.IMREAD_UNCHANGED)  # 알파 채널을 포함하여 로드

# 웹캠 캡처 객체 생성
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break  # 프레임을 제대로 읽지 못하면 루프 탈출

    # BGR 이미지를 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 얼굴 랜드마크 검출
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            h, w, _ = frame.shape
            
            myFace_1 = face_landmarks.landmark[280]
            myFace_2 = face_landmarks.landmark[459]
            # 두 눈 사이의 x, y 좌표 차이 계산
            delta_x = myFace_1.x - myFace_2.x
            delta_y = myFace_1.y - myFace_2.y

            # 두 눈이 이루는 각도 계산
            angle_radians = np.arctan2(delta_y, delta_x)  # 라디안 단위의 각도
            angle_degrees = np.degrees(angle_radians)  # 도 단위로 변환

            # 눈의 중심 위치 계산
            face_center_x = int((myFace_1.x + myFace_2.x) / 2 * w)
            face_center_y = int((myFace_1.y + myFace_2.y) / 2 * h)
            
            # 이미지 회전
            h, w = cat_face_img.shape[:2]
            center = (w // 2, h // 2)
            M = cv2.getRotationMatrix2D(center, -angle_degrees, 1)

            # # 회전할 때 이미지가 잘린다면 추가하세요
            # cos = np.abs(M[0, 0])
            # sin = np.abs(M[0, 1])

            # w = int((h * sin) + (w * cos))
            # h = int((h * cos) + (w * sin))

            # # 변환 행렬의 이동 부분 조정
            # M[0, 2] += (w / 2) - center[0]
            # M[1, 2] += (h / 2) - center[1]

            rotated_Cat_face_img = cv2.warpAffine(cat_face_img, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

            # 안경 이미지의 크기 조정 (여기서는 두 눈 사이의 거리를 기준으로 조정합니다)
            face_distance = int(np.linalg.norm(np.array([delta_x, delta_y]) * [w, h]))
            face_width = int(face_distance * 2.7) # 안경 크기가 너무 작게 표시된다면 이 값을 수정
            scale_factor = face_width / rotated_Cat_face_img.shape[1]
            resized_face = cv2.resize(rotated_Cat_face_img, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_AREA)

            # 안경 이미지의 알파 채널과 RGB 채널 분리
            face_alpha = resized_face[:, :, 3] / 255.0
            face_color = resized_face[:, :, :3]

            # 안경 위치 조정
            x = face_center_x - face_width // 2
            y = face_center_y - resized_face.shape[0] // 2

            # 알파 블렌딩을 통한 안경 이미지 합성
            for c in range(0, 3):
                frame[y:y+resized_face.shape[0], x:x+face_width, c] = face_alpha * face_color[:, :, c] + (1-face_alpha) * frame[y:y+resized_face.shape[0], x:x+face_width, c]

    # 결과 표시
    cv2.imshow('Glasses Overlay', frame)

    # 'q'를 누르면 루프 탈출
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [4]:
# 얼굴 감지 객체 설정 // 그리기 객체 초기화 // 그리기도 설정
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# 윕캠 객체 초기화 / 비디오 시작
cam = cv2.VideoCapture(0)

# 얼굴 감지 모델 설정
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
    while cam.isOpened():
        success, image = cam.read()
        if not success:
            print("웹캠에서 이미지를 읽어오는 데 실패했습니다.")
            # 비디오 캡처가 실패한 경우, 다음 프레임을 계속 시도합니다.
            continue    #아래 코드 무시
        
        # BGR 이미지를 RGB로 변환
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # # 성능 향상을 위해 이미지 쓰기 불가능으로 설정
        # image.flags.writeable = False

        # 이미지에서 얼굴 감지
        results = face_detection.process(image)
        print(results)

        # BGR 이미지를 RGB로 변환
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # # 이미지를 다시 BGR로 변환하여 그리기 작업 준비
        # image.flags.writeable = True

        # 감지된 얼굴에 대한 정보 처리 및 사각형 그리기
        if results.detections:
            for detection in results.detections:
                mp_drawing.draw_detection(image, detection)

        # 결과 이미지 표시
        cv2.imshow('Face Detection', image)

        # 'q'를 누르면 반복을 종료하고 창을 닫음
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

        # 모든 자원 해제
cam.release()
cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [35]:
# MediaPipe Face Mesh 모듈과 그리기 모듈 초기화
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# 웹캠 설정
cap = cv2.VideoCapture(0)
# MediaPipe 얼굴 메쉬 모델 로드
mp_face_mesh = mp.solutions.face_mesh

with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            continue

        # BGR 이미지를 RGB로 변환
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Face Mesh 처리
        results = face_mesh.process(image_rgb)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                
                # 얼굴의 외곽 윤곽 주변의 랜드마크 그리기
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_FACE_OVAL,  # 얼굴 외곽 랜드마크 연결
                    landmark_drawing_spec=None,  # 랜드마크 스타일 지정 (None이면 기본값 사용)
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=1)  # 연결선 스타일 지정
                )
                print(mp_face_mesh.FACEMESH_FACE_OVAL)
        # 결과 이미지 표시
        cv2.imshow('Eyes Landmarks', image)

        # 'q'를 눌러 종료
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

frozenset({(176, 149), (297, 332), (150, 136), (109, 10), (356, 454), (58, 132), (152, 148), (361, 288), (162, 21), (251, 389), (132, 93), (10, 338), (389, 356), (284, 251), (400, 377), (136, 172), (377, 152), (323, 361), (54, 103), (21, 54), (172, 58), (67, 109), (454, 323), (365, 379), (379, 378), (148, 176), (93, 234), (332, 284), (397, 365), (338, 297), (149, 150), (288, 397), (234, 127), (103, 67), (378, 400), (127, 162)})
frozenset({(176, 149), (297, 332), (150, 136), (109, 10), (356, 454), (58, 132), (152, 148), (361, 288), (162, 21), (251, 389), (132, 93), (10, 338), (389, 356), (284, 251), (400, 377), (136, 172), (377, 152), (323, 361), (54, 103), (21, 54), (172, 58), (67, 109), (454, 323), (365, 379), (379, 378), (148, 176), (93, 234), (332, 284), (397, 365), (338, 297), (149, 150), (288, 397), (234, 127), (103, 67), (378, 400), (127, 162)})
frozenset({(176, 149), (297, 332), (150, 136), (109, 10), (356, 454), (58, 132), (152, 148), (361, 288), (162, 21), (251, 389), (132, 93